In [1]:
from src.Experiments.TaxEffect import *
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

# Running model without tax
Set map-scheme based on your name:
- **Priyank**: top-heavy
- **Amish**: uniform
- **Ilia**: split

In [5]:
# Getting all scenarios
scenarios = load_scenarios("scenarios.csv")

## Set map-scheme based on your name
map_scheme = "uniform"

# Number of replicates and maximum steps, do not change
replicates = 30
max_steps = 200

# Running simulation
df = run_baseline(scenarios, map_scheme, replicates, max_steps)

# Saving the results
df.to_csv(f"TaxEffect/Base/results_{map_scheme}.csv", index=False)

100%|███████████████████████████████████████████████████████| 6/6 [00:22<00:00,  3.70s/it]


# Simulating with Tax
Select map-scheme based on previous step

In [16]:
# Getting all scenarios
scenarios = load_scenarios("scenarios.csv")

## Set map-scheme based on your name
map_scheme = "uniform"

# Number of replicates and maximum steps, do not change
replicates = 10
max_steps = 200

# Create tax systems
tax_systems = [
    ("progressive", "needs"),
    ("flat", "flat"),
    ("regressive", "random"),
    ("luxury", "progressive"),
    ("progressive", "progressive")
]

# Setting tax rates
tax_rates = [0.1, 0.25, 0.4]

# Running simulation
df = run_experiments(scenarios, map_scheme, tax_systems, tax_rates, replicates, max_steps)

# Saving the results
df.to_csv(f"TaxEffect/results_{map_scheme}.csv", index=False)

100%|█████████████████████████████████| 18/18 [00:08<00:00,  2.21it/s]


# Creating Time-Series

In [2]:
# Path to results and scenario
path_results = "TaxEffect"
scenarios = "scenarios.csv"

# Create folder for saving results
save_results = f"{path_results}/time_plots"
if not os.path.exists(save_results):
    os.makedirs(save_results)

# Load scenarios
scenarios = load_scenarios(scenarios)

# Looping through map schemes
map_schemes = ["uniform", "top_heavy", "split"]
with tqdm(total=len(map_schemes) * len(scenarios)) as pbar:
    for map_scheme in map_schemes:
        # Get the results of map scheme
        results = load_results(f"{path_results}/results_{map_scheme}.csv", scenarios)
        
        # Loop through scenarios
        for scenario_name, scenario in results.items():
            # Create time series plot
            fig, axs = time_series(scenario)
            
            # Add suptitle and apply tight layout
            fig.suptitle(f"{map_scheme.capitalize()} - {scenario_name.capitalize()}")
            fig.tight_layout()
            
            # Saving figure
            fig.savefig(f"{save_results}/{map_scheme}_{scenario_name}.png", dpi=300, bbox_inches="tight")
            
            # Close figure to avoid plotting
            plt.close(fig)
            
            # Update progress bar
            pbar.update(1)
    

100%|██████████| 18/18 [00:41<00:00,  2.31s/it]


# Creating Box-Plots

In [3]:
# Load scenarios
scenarios = load_scenarios("scenarios.csv")

# Map schemes
map_schemes = ["uniform", "top_heavy", "split"]

for map_scheme in map_schemes:
    # Get results
    results = load_results(f"TaxEffect/results_{map_scheme}.csv", scenarios)
    base_results = load_results(f"TaxEffect/Base/results_{map_scheme}.csv", scenarios)
    
    # Create boxplot
    figs = boxplot(results, base_results, map_scheme)
    
    for variable, (fig, ax) in figs.items():
        # Save figure
        fig.savefig(f"TaxEffect/boxplots/{map_scheme.capitalize()}_{variable.capitalize()}.png", dpi=300, bbox_inches="tight")
        
        # Close figure
        plt.close(fig)
        